![alt text](Imagens/7Labs.png "Title")

Esse código foi criado para realizar a coletar de dados de energia e água presentes nas planilhas de controle de dados da Utilidades e Meio Ambiente da Cervejaria Sete Lagoas no período de 2017 à 2020.

Primeiramente, realizamos a instalação e o import das bibliotecas necessárias

In [1]:
!pip install pandas
!pip install matplotlib
!pip install xlrd
!pip install openpyxl

In [2]:
import pandas as pd
import numpy as np
import h5py
import matplotlib.pyplot as plt
import datetime

Configuramos alguns parâmetros de exibição de imagens e tabelas.
Além disso, configuramos algumas variáveis necessárias para automatização do fluxo de coleta

In [3]:
plt.rcParams['figure.figsize'] = (5.0, 4.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

pd.set_option('max_rows', None)
pd.set_option('max_columns', None)

Files_Folder = 'Dados/'
Ano = ['2017','2018','2019','2020','2021']
Mes = ['Jan','Fev','Mar','Abr','Mai','Jun','Jul','Ago','Set','Out','Nov','Dez']

Realiza a leitura de todos os dados relativos à PL, EE e Consumo de Água no intervalo de 2017 à 2021

In [4]:
my_data_y_EE = pd.DataFrame()
my_data_y_MA = pd.DataFrame()
my_data_x = pd.DataFrame()

for j in range(0,4):
    for i in range(0,12):
        Files_path_MA = Files_Folder + 'MA/Água indice ' + Ano[j] +'.xlsx'
        Files_path_EE = Files_Folder + "EE/" + Ano[j] + '/Planilha de Dados ' + str(i+1) +'.xlsx'
    
        try:
            # Coleta informações das variáveis dependentes (EE)
            df_y_EE = pd.read_excel(Files_path_EE, sheet_name='EE_Dados', skiprows=[0,1,3], engine='openpyxl')
            df_y_EE_filter = df_y_EE[['Dia','PRINCIPAL (kW)']].copy()

            # Coleta informações das variáveis dependentes (MA)
            df_y_MA = pd.read_excel(Files_path_MA, sheet_name=Mes[i], skiprows=[0,1,2,4], engine='openpyxl')
            df_y_MA_filter = df_y_MA[['Dia','Consumo de Água (m³).1']].copy()
        
            # Coleta informações das variáveis independentes
            df_x = pd.read_excel(Files_path_EE, sheet_name='Prod. Líquida', skiprows=[0,1], engine='openpyxl')
            df_x_filter = df_x[['Dia','L501 (696)','L502 (697)','L503 (698)','L511 (699)','L512 (700)','L561 (927)',
                                'L562 (928)','563 (964)']].copy().fillna(0)

            
            my_data_y_EE = my_data_y_EE.append(df_y_EE_filter, ignore_index = True)
            my_data_y_MA = my_data_y_MA.append(df_y_MA_filter, ignore_index = True)
            my_data_x = my_data_x.append(df_x_filter, ignore_index = True)
        except Exception as e:
            print(e)

Remove NA e dados inconsistentes das bases de dados

In [5]:
my_data_y_EE = my_data_y_EE.dropna()
index = my_data_y_EE[(my_data_y_EE['Dia']=='Acc') | (my_data_y_EE['Dia']=='DIF.(%)') | (my_data_y_EE['Dia']=='Acum') | (my_data_y_EE['Dia']==1) | (my_data_y_EE['PRINCIPAL (kW)']==0)].index.values
my_data_y_EE.drop(index, inplace = True)

my_data_y_MA = my_data_y_MA.dropna()
index = my_data_y_MA[(my_data_y_MA['Dia']=='ACUM') | (my_data_y_MA['Dia']=='Meta acum') | (my_data_y_MA['Dia']=='VALOR REAL') | (my_data_y_MA['Dia']=='BASE HANNA')].index.values
my_data_y_MA.drop(index, inplace = True)

index = my_data_x[(my_data_x['Dia']=='Acc') | (my_data_x['Dia']=='DIF.(%)') | (my_data_x['Dia']=='Acum') | (my_data_x['Dia']==1) | (my_data_x['Dia']==0)].index.values
my_data_x.drop(index, inplace = True)

Ordena os dados de todas as planilhas segundo as datas dos dias em que ocorreram

In [6]:
my_data_y_EE['Dia'] = pd.to_datetime(my_data_y_EE['Dia'])
sorted_df_y_EE = my_data_y_EE.sort_values(by='Dia')

my_data_y_MA['Dia'] = pd.to_datetime(my_data_y_MA['Dia'])
sorted_df_y_MA = my_data_y_MA.sort_values(by='Dia')

my_data_x['Dia'] = pd.to_datetime(my_data_x['Dia'])
sorted_df_x = my_data_x.sort_values(by='Dia')

Faz uma união de todas as tabelas coletadas

In [7]:
my_data_y = my_data_y_EE.merge(my_data_y_MA, on='Dia', how='outer')
my_data = my_data_y.merge(my_data_x, on='Dia', how='outer')
sorted_df = my_data.sort_values(by='Dia')
sorted_df = sorted_df.dropna()
sorted_df

,Dia,PRINCIPAL (kW),Consumo de Água (m³).1,L501 (696),L502 (697),L503 (698),L511 (699),L512 (700),L561 (927),L562 (928),563 (964)
0,2017-01-01,1.250910e+05,5910,0,0.000,1938.816,0.0000,0.000,0.000,0.000,0.000
1,2017-01-02,1.953540e+05,6993,0,4278.120,3208.608,3308.4650,7322.891,1204.036,2999.520,13.028
2,2017-01-03,2.061260e+05,6521,4416,2113.560,2999.808,5926.3840,7916.885,759.917,5322.840,0.000
3,2017-01-04,2.143260e+05,5258,5166.72,2528.400,1741.824,6081.3630,8673.211,1204.036,3145.560,0.000
4,2017-01-05,2.107810e+05,7313,5583,1236.600,2426.112,0.0000,3193.487,1771.157,5055.240,0.000
5,2017-01-06,2.062780e+05,7260.3,3638.28,0.000,3157.128,0.0000,3859.566,2225.345,6852.360,0.000
6,2017-01-07,1.896190e+05,7404,3545.28,2196.240,2093.472,0.0000,4514.009,1315.890,4745.520,0.000
7,2017-01-08,5.527300e+04,2602.4,0,0.000,945.936,0.0000,0.000,0.000,0.000,0.000
8,2017-01-09,1.556050e+05,5140.3,0,1431.720,1059.696,1757.0800,2992.841,762.647,462.000,0.000
9,2017-01-10,1.025930e+05,2164,0,0.000,1233.792,0.0000,87.410,586.660,78.000,0.000


Grava os dados obtidos em um arquivo excel

In [8]:
sorted_df.to_excel(Files_Folder + 'DataBase.xlsx', index = False)

In [9]:
sorted_df.shape

(1558, 11)